In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Planetary Computer Tools for accessing and working with Planetary Computer data
import pystac
import pystac_client
import odc
import planetary_computer as pc
from odc.stac import stac_load
pc.settings.set_subscription_key('e9c294b01b2f483082124b8329af819f') # INSERT YOUR KEY HERE

# Requests for making HTTP requests
import requests

# Rich for creating rich-text tables
from rich.table import Table

# Itertools for creating an iterator cycling through a sequence indefinitely
from itertools import cycle

# TQDM for creating progress bars
from tqdm import tqdm

# Data Science
import pandas as pd
import numpy as np

## Response Variable

Before building the model, we need to load in the rice crop presence data. We have curated for you data from a certain region in Vietnam for the year 2020. The data consists of  geo locations (Latitude and Longitude) with a tag specifying if the crop present in each geo location is rice or not.  

In [2]:
crop_presence_data = pd.read_csv("Crop_Location_Data_20221201.csv")
#crop_presence_data = pd.read_csv("TEST_Crop_Location.csv")

crop_presence_data.head()

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


## Predictor Variables

<p align ="justify">Now that we have our crop location data, it is time to gather the predictor variables from the Sentinel-1 dataset. For a more in-depth look regarding the Sentinel-1 dataset and how to query it, see the Sentinel-1 <a href="https://challenge.ey.com/api/v1/storage/admin-files/6403146221623637-63ca8d537b1fe300146c79d0-Sentinel%201%20Phenology.ipynb/"> supplementary 
notebook</a>.
    

<p align = "justify">Sentinel-1 radar data penetrates through the clouds, thus helping us to get the band values with minimal atmospheric attenuation. Band values such as VV and VH help us in distinguishing between the rice and non rice crops. Hence we are choosing VV and VH as predictor variables for this experiment. 
        
<ul>
<li>VV - gamma naught values of signal transmitted with vertical polarization and received with vertical polarization with radiometric terrain correction applied.

<li>VH - gamma naught values of signal transmitted with vertical polarization and received with horizontal polarization with radiometric terrain correction applied.
</ul>

### Accessing the Sentinel-1 Data

<p align = "Justify">To get the Sentinel-1 data, we write a function called <i><b>get_sentinel_data.</b></i> This function will fetch VV and VH band values for a particular location over the specified time window. In this example, we have extracted VV and VH values for a day (21st March 2020). </p>

Explore the approach of building a bounding box (e.g., 5x5 pixels) around the given latitude and longitude positions and then extract the aggregated band values (e.g., average, median) to get normalized band values to build the model. Radar data has inherent variability at the pixel level due to variable scattering response from the target. This effect is called “speckle” and it is common to filter the data to smooth these variations. Try using a 3x3, 5x5 or 7x7 window around the specific latitude and longitude point to get improved results.

In [3]:
'''def get_sentinel_data(latlong, time_slice):
    
    #Returns VV and VH values for a given latitude and longitude 
    #Attributes:
    #latlong - A tuple with 2 elements - latitude and longitude
    #time_slice - Timeframe for which the VV and VH values have to be extracted
    

    latlong = latlong.replace('(', '').replace(')', '').replace(' ', '').split(',')
    lat, lon = map(float, latlong)
    
    box_size_deg = 0.0004  # Surrounding box in degrees, yields approximately 5x5 pixel region
    min_lon = lon - box_size_deg/2
    min_lat = lat - box_size_deg/2
    max_lon = lon + box_size_deg/2
    max_lat = lat + box_size_deg/2
    
    bbox_of_interest = (min_lon, min_lat, max_lon, max_lat)
    time_of_interest = time_slice

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    search = catalog.search(
        collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    items = list(search.get_all_items())
    
    # Define the pixel resolution for the final product
    # Define the scale according to our selected crs, so we will use degrees
    resolution = 10  # meters per pixel 
    scale = resolution / 111320.0  # degrees per pixel for crs=4326 
    
    # Load the data using Open Data Cube
    data = stac_load(items, bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox_of_interest, crs="EPSG:4326", resolution=scale)
    
    # Calculate the mean of the data across the sample region
    mean = data.mean(dim=['latitude', 'longitude']).compute()
    
    # Calculate RVI
    dop = (mean.vv / (mean.vv + mean.vh))
    m = 1 - dop
    rvi = (np.sqrt(dop)) * ((4 * mean.vh) / (mean.vv + mean.vh))

    # Calculate the averaged values
    vh_avg = mean.vh.mean().values.item()
    vv_avg = mean.vv.mean().values.item()
    rvi_avg = rvi.mean().values.item()
    
    return vh_avg, vv_avg, rvi_avg'''

'def get_sentinel_data(latlong, time_slice):\n    \n    #Returns VV and VH values for a given latitude and longitude \n    #Attributes:\n    #latlong - A tuple with 2 elements - latitude and longitude\n    #time_slice - Timeframe for which the VV and VH values have to be extracted\n    \n\n    latlong = latlong.replace(\'(\', \'\').replace(\')\', \'\').replace(\' \', \'\').split(\',\')\n    lat, lon = map(float, latlong)\n    \n    box_size_deg = 0.0004  # Surrounding box in degrees, yields approximately 5x5 pixel region\n    min_lon = lon - box_size_deg/2\n    min_lat = lat - box_size_deg/2\n    max_lon = lon + box_size_deg/2\n    max_lat = lat + box_size_deg/2\n    \n    bbox_of_interest = (min_lon, min_lat, max_lon, max_lat)\n    time_of_interest = time_slice\n\n    catalog = pystac_client.Client.open(\n        "https://planetarycomputer.microsoft.com/api/stac/v1"\n    )\n    search = catalog.search(\n        collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_in

In [4]:
def get_sentinel_data(latlong, time_slice):
    '''
    Returns VV and VH values for a given latitude and longitude 
    Attributes:
    latlong - A tuple with 2 elements - latitude and longitude
    time_slice - Timeframe for which the VV and VH values have to be extracted
    '''

    latlong = latlong.replace('(', '').replace(')', '').replace(' ', '').split(',')
    lat, lon = map(float, latlong)
    
    #box_size_deg = 0.0004  # Surrounding box in degrees, yields approximately 5x5 pixel region
    box_size_deg = 0.0125
    
    min_lon = lon - box_size_deg/2
    min_lat = lat - box_size_deg/2
    max_lon = lon + box_size_deg/2
    max_lat = lat + box_size_deg/2
    
    bbox_of_interest = (min_lon, min_lat, max_lon, max_lat)
    time_of_interest = time_slice

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    search = catalog.search(
        collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    items = list(search.get_all_items())
    
    # Define the pixel resolution for the final product
    # Define the scale according to our selected crs, so we will use degrees
    resolution = 10  # meters per pixel 
    scale = resolution / 111320.0  # degrees per pixel for crs=4326 
    
    # Load the data using Open Data Cube
    data = stac_load(items, bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox_of_interest, crs="EPSG:4326", resolution=scale)
    
    # Calculate the statistics of VH and VV
    vh_min = data.vh.min().values.item()
    vh_max = data.vh.max().values.item()
    vh_median = np.median(data.vh).item()

    vv_min = data.vv.min().values.item()
    vv_max = data.vv.max().values.item()
    vv_median = np.median(data.vv).item()

    # Calculate the mean of the data across the sample region
    mean = data.mean(dim=['latitude', 'longitude']).compute()
    
    # Calculate RVI
    dop = (mean.vv / (mean.vv + mean.vh))
    m = 1 - dop
    rvi = (np.sqrt(dop)) * ((4 * mean.vh) / (mean.vv + mean.vh))

    # Calculate additional RVI statistics
    rvi_min = rvi.min().values.item()
    rvi_max = rvi.max().values.item()
    rvi_median = np.median(rvi).item()
    
    # Calculate the averaged values
    vh_avg = mean.vh.mean().values.item()
    vv_avg = mean.vv.mean().values.item()
    rvi_avg = rvi.mean().values.item()
    
    return vh_avg, vv_avg, rvi_avg, rvi_min, rvi_max, rvi_median, vh_min, vh_max, vh_median, vv_min, vv_max, vv_median


In [5]:
## Function call to extract VV,VH Values
time_slice = "2020-03-20/2020-03-21"
assests = ['vh','vv','rvi']
vh_vv_rvi = []
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    vh_vv_rvi.append(get_sentinel_data(coordinates,time_slice))
vh_vv_data = pd.DataFrame(vh_vv_rvi, columns=['vh', 'vv', 'RVI', 'RVI(min)', 'RVI(max)', 'RVI(median)', 'vh(min)', 'vh(max)', 'vh(median)', 'vv(min)', 'vv(max)', 'vv(median)'])


100%|██████████| 600/600 [28:49<00:00,  2.88s/it]


## Joining the predictor variables and response variables
Now that we have extracted our predictor variables, we need to join them onto the response variable . We use the function <i><b>combine_two_datasets</b></i> to combine the predictor variables and response variables.The <i><b>concat</b></i> function from pandas comes in handy here.

In [6]:
def combine_two_datasets(dataset1,dataset2):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    data = pd.concat([dataset1,dataset2], axis=1)
    return data

In [7]:
crop_data = combine_two_datasets(crop_presence_data,vh_vv_data)
crop_data.head()

,Latitude and Longitude,Class of Land,vh,vv,RVI,RVI(min),RVI(max),RVI(median),vh(min),vh(max),vh(median),vv(min),vv(max),vv(median)
0,"(10.323727047081501, 105.2516346045924)",Rice,0.043734,0.205613,0.640923,0.587695,0.694151,0.640923,0.003147,3.314999,0.038026,0.015913,162.339966,0.148608
1,"(10.322364360592521, 105.27843410554115)",Rice,0.040909,0.168547,0.701644,0.665469,0.737819,0.701644,0.002306,1.885325,0.033830,0.007239,5.564671,0.143516
2,"(10.321455902933202, 105.25254306225168)",Rice,0.041375,0.203101,0.619992,0.577957,0.662027,0.619992,0.002989,3.314999,0.035738,0.015913,162.339966,0.145177
3,"(10.324181275911162, 105.25118037576274)",Rice,0.044391,0.206562,0.645607,0.593127,0.698088,0.645607,0.003147,3.314999,0.038637,0.016310,162.339966,0.149594
4,"(10.324635504740822, 105.27389181724476)",Rice,0.042551,0.174365,0.704493,0.655041,0.753944,0.704493,0.002744,0.467440,0.036355,0.012190,14.588789,0.147110


In [8]:
crop_data.to_csv('crop_data.csv', index=False)